In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import re

In [38]:
zones = ["", "China","emerg","Europe","Global","India","Japan","Rest"]

def process_df(x):
    columns = x.iloc[0,:]
    columns.name = ""
    x.columns = columns
    x = x.iloc[1:,:]
    x.reset_index(drop=True, names=columns, inplace=True)
    x.rename(index={"6":""}).head(7)
    return x

def get_betas(zone=""):
    path = "data_damodaran/discount_rate_estimation/beta_damodaran/totalbeta" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_unleveraged_betas(zone=""):
    path = "data_damodaran/discount_rate_estimation/beta_leverage/beta" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_taxrates(zone=""):
    path = "data_damodaran/discount_rate_estimation/tax/tax_rate_country/taxrate" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry name")
    return x

def get_wacc(zone=""):
    path = "data_damodaran/discount_rate_estimation/wacc_damodaran/wacc" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

In [50]:
x = yf.Ticker("NVDA")
x.info.get("industry")

sector = "Semiconductor"

### Capital Structure

In [36]:

E = x.balancesheet.loc["Stockholders Equity",:].iloc[0]
D = x.balancesheet.loc["Total Liabilities Net Minority Interest",:].iloc[0]

2023-01-31    19081000000.0
2022-01-31    17575000000.0
2021-01-31    11898000000.0
2020-01-31     5111000000.0
Name: Total Liabilities Net Minority Interest, dtype: object

### free-risk rate

In [54]:
get_wacc()

,Number of Firms,Beta,Cost of Equity,E/(D+E),Std Dev in Stock,Cost of Debt,Tax Rate,After-tax Cost of Debt,D/(D+E),Cost of Capital,Cost of Capital (Local Currency)
Industry Name,,,,,,,,,,,
Advertising,58,1.631738,0.135725,0.689708,0.527221,0.0588,0.063905,0.0441,0.310292,0.107295,0.107295
Aerospace/Defense,77,1.414182,0.122802,0.793291,0.375555,0.055,0.086028,0.04125,0.206709,0.105945,0.105945
Air Transport,21,1.415965,0.122908,0.349242,0.377275,0.055,0.104718,0.04125,0.650758,0.069769,0.069769
Apparel,39,1.324697,0.117487,0.659751,0.385087,0.055,0.120381,0.04125,0.340249,0.091547,0.091547
Auto & Truck,31,1.540598,0.130312,0.665819,0.526141,0.0588,0.029974,0.0441,0.334181,0.101501,0.101501
...,...,...,...,...,...,...,...,...,...,...,...
Trucking,35,1.545124,0.13058,0.694861,0.411713,0.055,0.147859,0.04125,0.305139,0.103322,0.103322
Utility (General),15,0.635139,0.076527,0.574135,0.149708,0.0473,0.13203,0.035475,0.425865,0.059045,0.059045
Utility (Water),16,1.152415,0.107253,0.697367,0.279613,0.055,0.084454,0.04125,0.302633,0.087279,0.087279


### Rd Damodaran Estimation

In [52]:
rd = get_wacc().loc[sector,"Cost of Debt"]

### Re Damodaran Estimation

In [56]:
re = get_wacc().loc[sector,"Cost of Equity"]

In [68]:
### 
x.incomestmt

,2023-01-31,2022-01-31,2021-01-31,2020-01-31
Tax Effect Of Unusual Items,-284130000.0,0.0,0.0,0.0
Tax Rate For Calcs,0.21,0.019,0.017,0.059
Normalized EBITDA,7340000000.0,11351000000.0,5691000000.0,3403000000.0
Total Unusual Items,-1353000000.0,NaN,NaN,NaN
Total Unusual Items Excluding Goodwill,-1353000000.0,NaN,NaN,NaN
Net Income From Continuing Operation Net Minority Interest,4368000000.0,9752000000.0,4332000000.0,2796000000.0
Reconciled Depreciation,1544000000.0,1174000000.0,1098000000.0,381000000.0
Reconciled Cost Of Revenue,11618000000.0,9439000000.0,6279000000.0,4150000000.0
EBITDA,5987000000.0,11351000000.0,5691000000.0,3403000000.0
EBIT,4443000000.0,10177000000.0,4593000000.0,3022000000.0
